Empty DataFrame
Columns: [Title, Source, Date, Description]
Index: []


Fetching article 1/5: https://news.google.com/rss/articles/CBMihAFBVV95cUxQR25fbmpFRU9uZzVHLUtCRzBNVEI1S3N3MGNNRHNTTnVQcFV0OXVKalppcVZxMF9FUjJQMmVqTWZxOFp4UnoxMnBTanV5UzJ2a0tFelhRZTU4bDI5bGI0SUhBQzZwbUJNV1FuTU8tWDBKQWlzNEdOUUY3X2xKRENMWnQwX1E?oc=5&hl=en-US&gl=US&ceid=US:en
Fetching article 2/5: https://news.google.com/rss/articles/CBMiigFBVV95cUxPWEtaeXVQdXlfaFdHajl2NzBJcG5oeWdxS2F4Y2x4WkxETmZqbGduNmdwUVIyNjdxZ3YyM2g5bXZoa3NHRDhULTJ1aW4yaTBUbFJVM2FNbGFXdFZlZXlGM09VM3IxSWo4dnJra0ppeVNNV3puZGtlVDZpb0ZiTEM3N2ozQl9waFVVT0E?oc=5&hl=en-US&gl=US&ceid=US:en
Fetching article 3/5: https://news.google.com/rss/articles/CBMilwFBVV95cUxQWnIta3Q0QmN1UHZJa0tQWGV3bjZkV2c3dk5NLWtHS0o2U0c2TGlyc3JIb2Z3bExhWnZkWXJ5WHhLcEpxMXJMdV9ZcXpfUUFaVVdSeWZOMjNucm9lVEwzbmM0WWxkVjVQZ1JyUFlFdXBpeWxKUnpDeHJ4amZobmRPYklJZXlmbmkzaFF4aGJyaTlvNVVRMGY0?oc=5&hl=en-US&gl=US&ceid=US:en
Fetching article 4/5: https://news.google.com/rss/articles/CBMimAFBVV95cUxPaDg0NWJ6a0JlTTR1eURHMUp5VFlSdk5qVklrY194RmlwbDBJVlhvWFFtYV8wTnhFcnBCUG

In [15]:
df[['title', 'full_text']].head()

,title,full_text
0,3 Top Artificial Intelligence (AI) Stocks Read...,Non-personalized content is influenced by thin...
1,3 Premier Artificial Intelligence (AI) Stocks ...,Non-personalized content is influenced by thin...
2,2025 Is the Year of BATMMAAN. Artificial Intel...,Non-personalized content is influenced by thin...
3,This Tech Stock Could Dominate the Market for ...,Non-personalized content is influenced by thin...
4,3 Premier Artificial Intelligence (AI) Stocks ...,Non-personalized content is influenced by thin...


In [16]:
df.iloc[0]['full_text']

'Non-personalized content is influenced by things like the content you’re currently viewing, activity in your active Search session, and your location. Non-personalized ads are influenced by the content you’re currently viewing and your general location. Personalized content and ads can also include more relevant results, recommendations, and tailored ads based on past activity from this browser, like previous Google searches. We also use cookies and data to tailor the experience to be age-appropriate, if relevant.'

In [19]:
from gnews import GNews
import pandas as pd
import trafilatura
import requests
import re
import time

def clean_text(text):
    if not text:
        return ""

    # Remove language selection menus
    text = re.sub(r'(?i)(English|Deutsch|Español|Français|Italiano).*?United States', '', text)

    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text)

    # Remove any remaining language codes
    text = re.sub(r'- [a-zA-Z]{2}(-[a-zA-Z]{2})?(\s|$)', '', text)

    return text.strip()

def get_full_news(query, max_articles=10):
    # Initialize GNews
    google_news = GNews(language='en', country='US', period='7d')
    google_news.max_results = max_articles

    # Get news articles
    news = google_news.get_news(query)
    news_df = pd.DataFrame(news)
    news_df['full_text'] = ''

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'DNT': '1'
    }

    for index, row in news_df.iterrows():
        try:
            print(f"Fetching article {index + 1}/{len(news_df)}: {row['url']}")

            # Get webpage content
            response = requests.get(row['url'], headers=headers, timeout=10)

            # Configure trafilatura settings
            downloaded = trafilatura.fetch_url(row['url'], headers=headers)

            if downloaded:
                text = trafilatura.extract(
                    downloaded,
                    include_comments=False,
                    include_tables=False,
                    no_fallback=False,
                    include_images=False,
                    include_links=False
                )

                if text:
                    cleaned_text = clean_text(text)
                    if len(cleaned_text) > 100:  # Only keep if substantial content
                        news_df.at[index, 'full_text'] = cleaned_text
                    else:
                        news_df.at[index, 'full_text'] = 'Content too short or invalid'
                else:
                    # Fallback: try direct HTML extraction
                    text = trafilatura.extract(
                        response.text,
                        include_comments=False,
                        include_tables=False,
                        no_fallback=False
                    )
                    if text:
                        news_df.at[index, 'full_text'] = clean_text(text)
                    else:
                        news_df.at[index, 'full_text'] = 'Failed to extract text'

            time.sleep(2)

        except Exception as e:
            print(f"Error processing article {index + 1}: {e}")
            news_df.at[index, 'full_text'] = f"Error: {str(e)}"
            continue

    # Remove rows with failed extractions
    news_df = news_df[~news_df['full_text'].str.contains('Failed to extract|Error:|Content too short')]

    return news_df

# Example usage
query = "artificial intelligence"
df = get_full_news(query, max_articles=5)

# Display results
for index, row in df.iterrows():
    print(f"\nArticle {index + 1}:")
    print(f"Title: {row['title']}")
    print(f"Text preview: {row['full_text'][:300]}...")  # Show first 300 chars
    print("-" * 80)

# Save to CSV (optional)
df.to_csv('ai_news.csv', index=False)

Fetching article 1/5: https://news.google.com/rss/articles/CBMi4wFBVV95cUxPUzlxRmtQXzcyUUljZGpDd0thVjI3UklLOU1DcFdNaWZwZE5xTWN5b283aEN1SFBRbVYyWXBJX2RDVG9MWVNwcmswV05PLXB0RG5CUE5Xb2I4NzFSSjJ5N2hjZ3Z1RlJKMG1xZ2cyaXRKUWN5MXpXRzBrS0JYYms0RDl3LWtRSFlQXzNEbV9ZTEwxaDdmVHdpcXJsWWxxdW9KX0tEMUpWak1RX3pRSU5OTFhucDFNZkViRGtCXzV0YTcwN25RUlVfWVVPTEhVTGwtWHZUY2NVWjl2MlpTb1NITDdVUQ?oc=5&hl=en-US&gl=US&ceid=US:en
Error processing article 1: fetch_url() got an unexpected keyword argument 'headers'
Fetching article 2/5: https://news.google.com/rss/articles/CBMirAFBVV95cUxNc3hseFhGNHZrWDNIbW0yQjNfMWpoWWJuZFVQNzNlQWtFVEs1OVhfN0ZJNmFHOUtOd3NhYjZSMm5CclVFam1BV1lCdmpxczdyVFdFNmxwODVDZkhRT3kxQmJUb0x2N201dFFMMHF1MFhsZnZfanZ4OEoyRzUwVWgxdzRYUFNva080Wm9YRUJPcnFVMzhab3BtM19kRGdZd3JUX1R4aTFTa3RiZDRw?oc=5&hl=en-US&gl=US&ceid=US:en
Error processing article 2: fetch_url() got an unexpected keyword argument 'headers'
Fetching article 3/5: https://news.google.com/rss/articles/CBMijwFBVV95cUxOV3pWa0xaWmFNMHdDVmsyX2hoWUN

In [4]:
json_resp[0]['url']

'https://news.google.com/rss/articles/CBMiigFBVV95cUxPWEtaeXVQdXlfaFdHajl2NzBJcG5oeWdxS2F4Y2x4WkxETmZqbGduNmdwUVIyNjdxZ3YyM2g5bXZoa3NHRDhULTJ1aW4yaTBUbFJVM2FNbGFXdFZlZXlGM09VM3IxSWo4dnJra0ppeVNNV3puZGtlVDZpb0ZiTEM3N2ozQl9waFVVT0E?oc=5&hl=en-US&gl=US&ceid=US:en'